### 여름학기 csv

1. pdf -> csv

In [132]:
import pdfplumber
import pandas as pd

# PDF 파일 경로
pdf_file_path = r"./libs/pdf/2024학년도 하계 계절수업 교과목 개설 현황표_공지용.pdf"

# 저장할 CSV 파일 경로 .\libs\csv
#csv_file_path = ".\libs\csv/kwu-24summer-timetable.csv"

# 데이터를 저장할 리스트 초기화
data = []

# PDF 파일 열기
with pdfplumber.open(pdf_file_path) as pdf:
    # 각 페이지 순회
    for page in pdf.pages:
        # 페이지에서 표 추출
        table = page.extract_table()
        if table:
            # 추출된 표 데이터를 리스트에 추가
            data.extend(table)

# 데이터가 없는 빈 행을 필터링
filtered_data = [row for row in data if any(cell.strip() for cell in row)]

# 추출된 데이터를 DataFrame으로 변환
df = pd.DataFrame(filtered_data[1:], columns=filtered_data[0])  # 첫 번째 행을 헤더로 사용

# DataFrame을 CSV 파일로 저장
#df.to_csv(csv_file_path, index=False)

#print("CSV 파일이 저장되었습니다.")


In [133]:
print(df)

    순번              학과/교양영역  구분                   과목명 학점 개설\n강좌수         강의시간  \
0    1             전체공통(정보)  교필                C프로그래밍  3       1        1,2교시   
1    2  전체공통\n(글로벌문화와제2외국어)  교선  초급중국어1\n(구,기초중국어회화1)  3       1          15일   
2    3  전체공통\n(글로벌문화와제2외국어)  교선            일본애니메이션과문화  3       1        3,4교시   
3    4  전체공통\n(글로벌문화와제2외국어)  교선                초급일본어1  3       1        3,4교시   
4    5          전체공통(예술과체육)  교선               대중음악의역사  3       1          15일   
..  ..                  ...  ..                   ... ..     ...          ...   
56  57                 전체공통  교선          전략적의사결정과문제해결  3       1    광운대학교 박종구   
57  58                 전체공통  교선               행복한금융생활  3       1    광운대학교 송영출   
58  59                 전체공통  교선     화폐와금융의과거,현재,그리고미래  3       1    한양대학교 현정환   
59  60                 전체공통  교선               음식과세계문화  3       1  덕성여자대학교 김건희   
60  61                 전체공통  교선               그린IT의이해  3       1    한성대학교 조세홍   

      직종  담당교수             

2. 칼럼 '연도', '학기' 칼럼 추가
3. '개설강좌수', '직종' 칼럼 삭제

In [134]:
# '연도'와 '학기' 칼럼 추가
df.insert(1, '연도', '24')  # '순번' 뒤에 '연도' 칼럼 추가
df.insert(2, '학기', 'summer')  # '연도' 뒤에 '학기' 칼럼 추가

# '개설강좌수'와 '직종' 칼럼 삭제
df = df.drop(columns=['개설\n강좌수', '직종'])

# 결과 출력
print(df)

    순번  연도      학기              학과/교양영역  구분                   과목명 학점  \
0    1  24  summer             전체공통(정보)  교필                C프로그래밍  3   
1    2  24  summer  전체공통\n(글로벌문화와제2외국어)  교선  초급중국어1\n(구,기초중국어회화1)  3   
2    3  24  summer  전체공통\n(글로벌문화와제2외국어)  교선            일본애니메이션과문화  3   
3    4  24  summer  전체공통\n(글로벌문화와제2외국어)  교선                초급일본어1  3   
4    5  24  summer          전체공통(예술과체육)  교선               대중음악의역사  3   
..  ..  ..     ...                  ...  ..                   ... ..   
56  57  24  summer                 전체공통  교선          전략적의사결정과문제해결  3   
57  58  24  summer                 전체공통  교선               행복한금융생활  3   
58  59  24  summer                 전체공통  교선     화폐와금융의과거,현재,그리고미래  3   
59  60  24  summer                 전체공통  교선               음식과세계문화  3   
60  61  24  summer                 전체공통  교선               그린IT의이해  3   

           강의시간  담당교수               비 고  
0         1,2교시   정진범                    
1           15일   곡효운  원격강의 (사전녹화 100%)  
2        

In [135]:
df.columns

Index(['순번', '연도', '학기', '학과/교양영역', '구분', '과목명', '학점', '강의시간', '담당교수', '비 고'], dtype='object')

4. [수동확인 필요] '학과/교양영역' 칼럼 3개로 분리 (학과 : cantakeBunBan / 학과 : majorRecogBunBan), (교양영역 : lecTheme)

In [136]:
# '학과/교양영역' 칼럼을 세 개의 새로운 칼럼으로 분리
def split_major_lec(row):
    lecTheme = ''
    cantakeBunBan = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
    majorRecogBunBan = ''
    
    # 괄호 안의 내용을 교양영역으로 설정
    if '(' in row and ')' in row:
        lecTheme = row[row.find('(')+1:row.find(')')]

    # 전체공통에 따른 분리 !! [계절마다 추가해야할 수도 있음]
    if '전체공통' in row:
        cantakeBunBan = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
        majorRecogBunBan = ''
    else:
        cantakeBunBan = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
        if '국제통상학부' in row:
            majorRecogBunBan = 'B5,B6'
        elif '법학부' in row:
            majorRecogBunBan = 'L1,L2,L3'
        elif '미디어커뮤니케이션학부' in row:
            majorRecogBunBan = 'M1,M2'
        elif '동북아문화산업학부' in row:
            majorRecogBunBan = 'R4,R5'
        elif '소프트웨어학부' in row:
            majorRecogBunBan = 'C4,C5,C6'
    
    return cantakeBunBan, majorRecogBunBan, lecTheme

df[['수강가능한 학과', '전공 인정 학과 ', '테마']] = df['학과/교양영역'].apply(lambda x: pd.Series(split_major_lec(x)))

df

,순번,연도,학기,학과/교양영역,구분,과목명,학점,강의시간,담당교수,비 고,수강가능한 학과,전공 인정 학과,테마
0,1,24,summer,전체공통(정보),교필,C프로그래밍,3,"1,2교시",정진범,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,정보
1,2,24,summer,전체공통\n(글로벌문화와제2외국어),교선,"초급중국어1\n(구,기초중국어회화1)",3,15일,곡효운,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어
2,3,24,summer,전체공통\n(글로벌문화와제2외국어),교선,일본애니메이션과문화,3,"3,4교시",유양근,원격강의 (사전녹화 87%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어
3,4,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급일본어1,3,"3,4교시",김희성,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어
4,5,24,summer,전체공통(예술과체육),교선,대중음악의역사,3,15일,김철수,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,57,24,summer,전체공통,교선,전략적의사결정과문제해결,3,광운대학교 박종구,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,
57,58,24,summer,전체공통,교선,행복한금융생활,3,광운대학교 송영출,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,
58,59,24,summer,전체공통,교선,"화폐와금융의과거,현재,그리고미래",3,한양대학교 현정환,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,
59,60,24,summer,전체공통,교선,음식과세계문화,3,덕성여자대학교 김건희,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,


5. [수동확인필요] 연계전공은 대학원을 연계해서 듣는 수업을 의미, 따라서 이수구분이 '연선', '연필' 과목 row는 삭제

In [137]:
# 필터링 전의 행 수
initial_count = len(df)

# '구분'이 '연선'이나 '연필'인 행을 제외하고 필터링
df_filtered = df[~df['구분'].isin(['연선', '연필'])]

# 필터링 후의 행 수
final_count = len(df_filtered)

# 삭제된 행의 수
deleted_count = initial_count - final_count

# 결과 출력
print(f"삭제된 행의 수: {deleted_count}")

삭제된 행의 수: 1


6. 강의시간표에 이러닝('대학교'가 있다면) 이러닝_담당교수으로 다 옮기기

In [138]:
import numpy as np

df = df_filtered

# '이러닝_담당교수'라는 새로운 칼럼 생성
df['이러닝_담당교수'] = df['강의시간'].apply(lambda x: x if '대학교' in x else '')

# '강의시간' 칼럼에서 '대학교'가 포함된 텍스트 제거
df['강의시간'] = df['강의시간'].apply(lambda x: '' if '대학교' in x else x)

# 만약 NaN 값이 생긴 경우 빈 문자열로 대체
df = df.replace(np.nan, '', regex=True)

# 결과 출력
print(df)

    순번  연도      학기              학과/교양영역  구분                   과목명 학점  \
0    1  24  summer             전체공통(정보)  교필                C프로그래밍  3   
1    2  24  summer  전체공통\n(글로벌문화와제2외국어)  교선  초급중국어1\n(구,기초중국어회화1)  3   
2    3  24  summer  전체공통\n(글로벌문화와제2외국어)  교선            일본애니메이션과문화  3   
3    4  24  summer  전체공통\n(글로벌문화와제2외국어)  교선                초급일본어1  3   
4    5  24  summer          전체공통(예술과체육)  교선               대중음악의역사  3   
5    6  24  summer          전체공통(예술과체육)  교선               생활속의디자인  3   
6    7  24  summer             전체공통(수학)  교선                 공학수학1  3   
7    8  24  summer             전체공통(수학)  교선                 공학수학1  3   
8    9  24  summer             전체공통(수학)  교선                 공학수학2  3   
9   10  24  summer             전체공통(수학)  교선                 공학수학2  3   
10  11  24  summer             전체공통(수학)  교선               기초수학및연습  3   
11  12  24  summer             전체공통(수학)  교선              대학수학및연습1  3   
12  13  24  summer             전체공통(수학)  교선              대학수학및연습

C:\Users\user\AppData\Local\Temp\ipykernel_5580\3298731854.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['이러닝_담당교수'] = df['강의시간'].apply(lambda x: x if '대학교' in x else '')
C:\Users\user\AppData\Local\Temp\ipykernel_5580\3298731854.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['강의시간'] = df['강의시간'].apply(lambda x: '' if '대학교' in x else x)


In [139]:
df

,순번,연도,학기,학과/교양영역,구분,과목명,학점,강의시간,담당교수,비 고,수강가능한 학과,전공 인정 학과,테마,이러닝_담당교수
0,1,24,summer,전체공통(정보),교필,C프로그래밍,3,"1,2교시",정진범,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,정보,
1,2,24,summer,전체공통\n(글로벌문화와제2외국어),교선,"초급중국어1\n(구,기초중국어회화1)",3,15일,곡효운,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,
2,3,24,summer,전체공통\n(글로벌문화와제2외국어),교선,일본애니메이션과문화,3,"3,4교시",유양근,원격강의 (사전녹화 87%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,
3,4,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급일본어1,3,"3,4교시",김희성,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,
4,5,24,summer,전체공통(예술과체육),교선,대중음악의역사,3,15일,김철수,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육,
5,6,24,summer,전체공통(예술과체육),교선,생활속의디자인,3,"4,5교시",김문석,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육,
6,7,24,summer,전체공통(수학),교선,공학수학1,3,"1,2교시",장정환,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,
7,8,24,summer,전체공통(수학),교선,공학수학1,3,"5,6교시",최현옥,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,
8,9,24,summer,전체공통(수학),교선,공학수학2,3,"3,4교시",윤미,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,
9,10,24,summer,전체공통(수학),교선,공학수학2,3,"5,6교시",안소영,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,


7. '기타' 칼럼을 만들어서 구과목명에 대한 정보 담기
    새로운 칼럼인 '기타' 생성
    '(구,)' 텍스트가 있는 경우 기타로 '(' 칼럼부터 옮기기

In [140]:
# 기타 칼럼 생성
df['기타'] = df['과목명'].str.extract(r'(\(구,.*\))')

# 강의명 칼럼에서 기타 부분 제거
df['과목명'] = df['과목명'].str.replace(r'\(구,.*?\)', '', regex=True).str.strip()

df  

,순번,연도,학기,학과/교양영역,구분,과목명,학점,강의시간,담당교수,비 고,수강가능한 학과,전공 인정 학과,테마,이러닝_담당교수,기타
0,1,24,summer,전체공통(정보),교필,C프로그래밍,3,"1,2교시",정진범,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,정보,,NaN
1,2,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급중국어1,3,15일,곡효운,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,,"(구,기초중국어회화1)"
2,3,24,summer,전체공통\n(글로벌문화와제2외국어),교선,일본애니메이션과문화,3,"3,4교시",유양근,원격강의 (사전녹화 87%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,,NaN
3,4,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급일본어1,3,"3,4교시",김희성,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,,NaN
4,5,24,summer,전체공통(예술과체육),교선,대중음악의역사,3,15일,김철수,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육,,NaN
5,6,24,summer,전체공통(예술과체육),교선,생활속의디자인,3,"4,5교시",김문석,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육,,NaN
6,7,24,summer,전체공통(수학),교선,공학수학1,3,"1,2교시",장정환,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,,NaN
7,8,24,summer,전체공통(수학),교선,공학수학1,3,"5,6교시",최현옥,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,,NaN
8,9,24,summer,전체공통(수학),교선,공학수학2,3,"3,4교시",윤미,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,,NaN
9,10,24,summer,전체공통(수학),교선,공학수학2,3,"5,6교시",안소영,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,,NaN


In [141]:
df.to_csv(".\libs\csv/kwu-24summer-timetable_v1.csv") # v1은 csv를 통해 만들어진 것을 의미 

8. 유의사항

    1. 학과/교양영역 -> 칼럼 검증을 위해 남겨둔거지 삭제해두됨
    
    2. 비고 -> 강의계획서로 채울거임(원격강의(%정도), e러닝 유무) 삭제해도됨

9. 강의계획서 크롤링 csv와 concat 하기
학정번호(lecNumber) 강의시간(lecTime)  강의시수(lecWeekTime)   강의실(lecClassroom)  비고(원격강의)  비고(e러닝 유무)

+ 이외에 강의계획서를 통해 뽑을 수 있는 모든 것들 

In [142]:
### kwu-24summer-timetable_v2 : csv 칼럼화 + 강의계획서 크롤링(***) 합치기 + 에타강의평은 학정번호 같은걸로 연결시키기 

교과목명	년도학기	이수구분	학정번호	담당교수	강의소개	학점/시간	강의시간 및 강의실	연락처	이동전화	이메일	담당조교	장애학생 지원	교과목 개요	대표역량	학습목표 및 학습방법	학습성과	영어강의 비율	출석	중간고사	기말고사	과제보고서	수업태도	Quiz	기타1	영어강의	중국어 강의	일본어 강의	TBL강의	PBL강의	세미나강의	소규모강의	융합강의	팀티칭강의	일학습병행강의	외국인전용강의	서울권역e-러닝	실험실습실기설계강의	집중이수제강의	원격수업 (100%)	원격수업 (50% 이상)	100%대면강의	대면+실시간 화상강의	대면+사전녹화강의	대면+실시간 화상강의+사전녹화강의	실시간 화상강의+사전녹화강의	100%실시간 화상강의	100%사전녹화강의	실험/실습 실기	외국어 강의	원격수업	TBL강의	PBL강의	토론식 수업(세미나 포함)프로젝트	실기(예체능)	기타	수강일정	수강일정2	리스트	에러


-> 학정번호, 강의시간 및 강의실 -> 강의시간/강의실, 학점/시간 -> 시수
-> +) 강의소개 연락처	이동전화	이메일	담당조교	장애학생 지원	교과목 개요	대표역량	학습목표 및 학습방법	학습성과	영어강의 비율	출석	중간고사	기말고사	과제보고서	수업태도	Quiz	기타1	영어강의	중국어 강의	일본어 강의	TBL강의	PBL강의	세미나강의	소규모강의	융합강의	팀티칭강의	일학습병행강의	외국인전용강의	서울권역e-러닝	실험실습실기설계강의	집중이수제강의	원격수업 (100%)	원격수업 (50% 이상)	100%대면강의	대면+실시간 화상강의	대면+사전녹화강의	대면+실시간 화상강의+사전녹화강의	실시간 화상강의+사전녹화강의	100%실시간 화상강의	100%사전녹화강의	실험/실습 실기	외국어 강의	원격수업	TBL강의	PBL강의	토론식 수업(세미나 포함)프로젝트	실기(예체능)	기타	수강일정	수강일정2	리스트	에러

1. 강의시간 및 강의실 분리  
()안은 강의실을 의미, ()안의 내용이 중복되면 한번만 가져오기 -> '강의실' 칼럼을 새로 만들어 넣어놓기
이외의 텍스트는 '강의시간' 칼럼을 새로 만들어 넣어놓기

2. 학점/시간 중 시간만 분리
'-' 이후의 숫자인 시간을 분리

3. 교과목명, 년도학기, 이수구분, 담당교수, 학점/시간, 강의시간 및 강의실을 제외하고 위의 csv와 concat하기 

In [143]:
import re
import pandas as pd

# CSV 파일을 읽어올 때 인코딩 지정
df = pd.read_csv(r"D:/LectureRecommendationService/data_preprocessing/[강의시간표] kwu-lecture-timetable/24-여름학기/libs/csv/lecture_plan.csv", encoding='cp949')

def split_time_and_room(text):
    if isinstance(text, float):  # 만약 float가 들어온다면 빈 문자열로 처리
        text = ''
    
    # 괄호 안의 내용을 모두 추출
    rooms = re.findall(r'\((.*?)\)', text)
    # 중복된 강의실 제거
    unique_rooms = list(set(rooms))

    # 괄호와 괄호 안의 내용을 제거하여 강의시간만 추출
    times = re.sub(r'\s*\(.*?\)', '', text).strip()

    return times, ', '.join(unique_rooms)

# '강의시간 및 강의실' 열의 NaN 값을 빈 문자열로 변환
df['강의시간 및 강의실'] = df['강의시간 및 강의실'].fillna('')

# 새로운 칼럼에 강의시간과 강의실 정보를 저장
df['강의시간'], df['강의실'] = zip(*df['강의시간 및 강의실'].apply(split_time_and_room))

# 결과 출력
print(df[['강의시간', '강의실']])


                                   강의시간     강의실
0           월 12교시 화 12교시 수 12교시 목 12교시    연204
1                                 토 1교시     미지정
2           월 34교시 화 34교시 수 34교시 목 34교시   한울304
3           월 34교시 화 34교시 수 34교시 목 34교시   한울307
4                                 토 2교시     미지정
5           월 45교시 화 45교시 수 45교시 목 45교시   한울310
6           월 12교시 화 12교시 수 12교시 목 12교시    비211
7           월 56교시 화 56교시 수 56교시 목 56교시    비211
8           월 34교시 화 34교시 수 34교시 목 34교시    비409
9           월 56교시 화 56교시 수 56교시 목 56교시    비409
10          월 56교시 화 56교시 수 56교시 목 56교시    비519
11          월 12교시 화 12교시 수 12교시 목 12교시    비421
12          월 34교시 화 34교시 수 34교시 목 34교시    비421
13          월 56교시 화 56교시 수 56교시 목 56교시    비421
14          월 12교시 화 12교시 수 12교시 목 12교시    비520
15          월 56교시 화 56교시 수 56교시 목 56교시    비520
16          월 12교시 화 12교시 수 12교시 목 12교시  한울B103
17          월 12교시 화 12교시 수 12교시 목 12교시    비619
18          월 34교시 화 34교시 수 34교시 목 34교시   한울502
19          월 45교시 화 45교시 수 45교시 목 45교시 

In [144]:
import pandas as pd

# 요일과 시간 매핑 사전
day_mapping = {'월': 1, '화': 2, '수': 3, '목': 4, '금': 5, '토': 6}

# 교시를 숫자로 변환하는 함수
def map_time_period(period):
    try:
        # 교시에서 시작값과 끝값 추출
        start = int(period[0])
        end = int(period[-1])
        if start == end:  # 연강이 아닌 경우
            return f"{start}"
        else:  # 연강인 경우
            return f"{start},{end}"
    except Exception as e:
        print(f"Error processing period '{period}': {e}")
        return ''  # 빈 문자열 반환

# 강의시간 컬럼을 변환하는 함수
def convert_schedule(schedule):
    try:
        items = schedule.split()
        converted = []
        for i in range(0, len(items), 2):
            day = day_mapping.get(items[i], None)
            if day is not None and i + 1 < len(items):
                times = map_time_period(items[i + 1].replace('교시', ''))
                if times:  # 빈 문자열이 아닌 경우에만 추가
                    converted.append(f"({day}:{times})")
        return ','.join(converted)
    except Exception as e:
        print(f"Error processing schedule '{schedule}': {e}")
        return ''  # 빈 문자열 반환

# '강의시간' 컬럼에 대해 변환 수행
df['강의시간 (요일,시간)'] = df['강의시간'].apply(convert_schedule)

# 결과 출력
print(df[['강의시간', '강의시간 (요일,시간)']])


                                   강의시간                     강의시간 (요일,시간)
0           월 12교시 화 12교시 수 12교시 목 12교시  (1:1,2),(2:1,2),(3:1,2),(4:1,2)
1                                 토 1교시                            (6:1)
2           월 34교시 화 34교시 수 34교시 목 34교시  (1:3,4),(2:3,4),(3:3,4),(4:3,4)
3           월 34교시 화 34교시 수 34교시 목 34교시  (1:3,4),(2:3,4),(3:3,4),(4:3,4)
4                                 토 2교시                            (6:2)
5           월 45교시 화 45교시 수 45교시 목 45교시  (1:4,5),(2:4,5),(3:4,5),(4:4,5)
6           월 12교시 화 12교시 수 12교시 목 12교시  (1:1,2),(2:1,2),(3:1,2),(4:1,2)
7           월 56교시 화 56교시 수 56교시 목 56교시  (1:5,6),(2:5,6),(3:5,6),(4:5,6)
8           월 34교시 화 34교시 수 34교시 목 34교시  (1:3,4),(2:3,4),(3:3,4),(4:3,4)
9           월 56교시 화 56교시 수 56교시 목 56교시  (1:5,6),(2:5,6),(3:5,6),(4:5,6)
10          월 56교시 화 56교시 수 56교시 목 56교시  (1:5,6),(2:5,6),(3:5,6),(4:5,6)
11          월 12교시 화 12교시 수 12교시 목 12교시  (1:1,2),(2:1,2),(3:1,2),(4:1,2)
12          월 34교시 화 34교시 수 34교시 목 34교시  (1:3,4),(2

In [145]:
df

,교과목명,년도학기,이수구분,학정번호,담당교수,강의소개,학점/시간,강의시간 및 강의실,연락처,이동전화,...,토론식 수업(세미나 포함)프로젝트,실기(예체능),기타,수강일정,수강일정2,리스트,에러,강의시간,강의실,"강의시간 (요일,시간)"
0,C프로그래밍-C Programming,2024/3,NaN,0000-1-0019-01,NaN,NaN,3/3,월 12교시 (연204) 화 12교시 (연204) 수 12교시 (연204) 목 12...,01056175737,010-5617-5737,...,unchecked,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 2...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 12교시 화 12교시 수 12교시 목 12교시,연204,"(1:1,2),(2:1,2),(3:1,2),(4:1,2)"
1,초급중국어1-Beginning Chinese 1,2024/3,교선,0000-1-2950-01,NaN,NaN,3/3,토 1교시 (미지정),9408366,01086462826,...,unchecked,unchecked,unchecked,"주차 강의내용 강의운영방식\n1 중국어에 관한 기본 지식, 중국어 발음\n2 중국어...","<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,토 1교시,미지정,(6:1)
2,일본애니메이션과문화-Japanese animation and Culture,2024/3,교선,0000-1-7363-01,NaN,NaN,3/3,월 34교시 (한울304) 화 34교시 (한울304) 수 34교시 (한울304) 목...,010-6302-0608,010-6302-0608,...,unchecked,unchecked,unchecked,주차 강의내용 강의운영방식\n1 일본 애니메이션의 역사와 현황\n사전 녹화 동영상 ...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,월 34교시 화 34교시 수 34교시 목 34교시,한울304,"(1:3,4),(2:3,4),(3:3,4),(4:3,4)"
3,초급일본어1-Elementary Japanese 1,2024/3,교선,0000-1-3593-01,NaN,NaN,3/3,월 34교시 (한울307) 화 34교시 (한울307) 수 34교시 (한울307) 목...,NaN,010-8000-5069,...,unchecked,unchecked,unchecked,주차 강의내용 강의운영방식\n1 강의 개요 \n 제1과 일본어의 문자와 오십음도\n...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,월 34교시 화 34교시 수 34교시 목 34교시,한울307,"(1:3,4),(2:3,4),(3:3,4),(4:3,4)"
4,대중음악의역사-History of Pop Music,2024/3,교선,0000-2-6948-01,NaN,NaN,3/3,토 2교시 (미지정),NaN,02-940-8109,...,checked,unchecked,unchecked,"주차 강의내용 강의운영방식\n1 오리엔테이션 수업 전반에 대한 PREVIEW, 과제...","<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,토 2교시,미지정,(6:2)
5,생활속의디자인-Introduction to Design,2024/3,교선,0000-1-5682-01,NaN,NaN,3/3,월 45교시 (한울310) 화 45교시 (한울310) 수 45교시 (한울310) 목...,02-940-5545,010-6208-5949,...,unchecked,unchecked,unchecked,주차 강의내용 강의운영방식\n1 디자인이란\n2 형태의 지각과 디자인의 원리\n3 ...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,월 45교시 화 45교시 수 45교시 목 45교시,한울310,"(1:4,5),(2:4,5),(3:4,5),(4:4,5)"
6,공학수학1-Engineering Mathematics 1,2024/3,NaN,0000-2-4969-01,NaN,NaN,3/3,월 12교시 (비211) 화 12교시 (비211) 수 12교시 (비211) 목 12...,02 940 5220,010-3255-1343,...,unchecked,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 12교시 화 12교시 수 12교시 목 12교시,비211,"(1:1,2),(2:1,2),(3:1,2),(4:1,2)"
7,공학수학1-Engineering Mathematics 1,2024/3,NaN,0000-2-4969-02,NaN,NaN,3/3,월 56교시 (비211) 화 56교시 (비211) 수 56교시 (비211) 목 56...,NaN,010-2807-3974,...,unchecked,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 56교시 화 56교시 수 56교시 목 56교시,비211,"(1:5,6),(2:5,6),(3:5,6),(4:5,6)"
8,공학수학2-Engineering Mathematics 2,2024/3,NaN,0000-2-4970-01,NaN,NaN,3/3,월 34교시 (비409) 화 34교시 (비409) 수 34교시 (비409) 목 34...,NaN,01024224450,...,unchecked,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 34교시 화 34교시 수 34교시 목 34교시,비409,"(1:3,4),(2:3,4),(3:3,4),(4:3,4)"
9,공학수학2-Engineering Mathematics 2,2024/3,NaN,0000-2-4970-02,NaN,NaN,3/3,월 56교시 (비409) 화 56교시 (비409) 수 56교시 (비409) 목 56...,NaN,01038605494,...,unchecked,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 56교시 화 56교시 수 56교시 목 56교시,비409,"(1:5,6),(2:5,6),(3:5,6),(4:5,6)"


In [146]:
### 2. 학점/시간 중 시수만 분리

# '시간' 칼럼 생성: '/' 바로 뒤의 텍스트만 추출
df['시간'] = df['학점/시간'].apply(lambda x: str(x).split('/')[-1] if pd.notnull(x) else None)

# 결과 출력
df

,교과목명,년도학기,이수구분,학정번호,담당교수,강의소개,학점/시간,강의시간 및 강의실,연락처,이동전화,...,실기(예체능),기타,수강일정,수강일정2,리스트,에러,강의시간,강의실,"강의시간 (요일,시간)",시간
0,C프로그래밍-C Programming,2024/3,NaN,0000-1-0019-01,NaN,NaN,3/3,월 12교시 (연204) 화 12교시 (연204) 수 12교시 (연204) 목 12...,01056175737,010-5617-5737,...,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 2...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 12교시 화 12교시 수 12교시 목 12교시,연204,"(1:1,2),(2:1,2),(3:1,2),(4:1,2)",3
1,초급중국어1-Beginning Chinese 1,2024/3,교선,0000-1-2950-01,NaN,NaN,3/3,토 1교시 (미지정),9408366,01086462826,...,unchecked,unchecked,"주차 강의내용 강의운영방식\n1 중국어에 관한 기본 지식, 중국어 발음\n2 중국어...","<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,토 1교시,미지정,(6:1),3
2,일본애니메이션과문화-Japanese animation and Culture,2024/3,교선,0000-1-7363-01,NaN,NaN,3/3,월 34교시 (한울304) 화 34교시 (한울304) 수 34교시 (한울304) 목...,010-6302-0608,010-6302-0608,...,unchecked,unchecked,주차 강의내용 강의운영방식\n1 일본 애니메이션의 역사와 현황\n사전 녹화 동영상 ...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,월 34교시 화 34교시 수 34교시 목 34교시,한울304,"(1:3,4),(2:3,4),(3:3,4),(4:3,4)",3
3,초급일본어1-Elementary Japanese 1,2024/3,교선,0000-1-3593-01,NaN,NaN,3/3,월 34교시 (한울307) 화 34교시 (한울307) 수 34교시 (한울307) 목...,NaN,010-8000-5069,...,unchecked,unchecked,주차 강의내용 강의운영방식\n1 강의 개요 \n 제1과 일본어의 문자와 오십음도\n...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,월 34교시 화 34교시 수 34교시 목 34교시,한울307,"(1:3,4),(2:3,4),(3:3,4),(4:3,4)",3
4,대중음악의역사-History of Pop Music,2024/3,교선,0000-2-6948-01,NaN,NaN,3/3,토 2교시 (미지정),NaN,02-940-8109,...,unchecked,unchecked,"주차 강의내용 강의운영방식\n1 오리엔테이션 수업 전반에 대한 PREVIEW, 과제...","<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,토 2교시,미지정,(6:2),3
5,생활속의디자인-Introduction to Design,2024/3,교선,0000-1-5682-01,NaN,NaN,3/3,월 45교시 (한울310) 화 45교시 (한울310) 수 45교시 (한울310) 목...,02-940-5545,010-6208-5949,...,unchecked,unchecked,주차 강의내용 강의운영방식\n1 디자인이란\n2 형태의 지각과 디자인의 원리\n3 ...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN,월 45교시 화 45교시 수 45교시 목 45교시,한울310,"(1:4,5),(2:4,5),(3:4,5),(4:4,5)",3
6,공학수학1-Engineering Mathematics 1,2024/3,NaN,0000-2-4969-01,NaN,NaN,3/3,월 12교시 (비211) 화 12교시 (비211) 수 12교시 (비211) 목 12...,02 940 5220,010-3255-1343,...,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 12교시 화 12교시 수 12교시 목 12교시,비211,"(1:1,2),(2:1,2),(3:1,2),(4:1,2)",3
7,공학수학1-Engineering Mathematics 1,2024/3,NaN,0000-2-4969-02,NaN,NaN,3/3,월 56교시 (비211) 화 56교시 (비211) 수 56교시 (비211) 목 56...,NaN,010-2807-3974,...,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 56교시 화 56교시 수 56교시 목 56교시,비211,"(1:5,6),(2:5,6),(3:5,6),(4:5,6)",3
8,공학수학2-Engineering Mathematics 2,2024/3,NaN,0000-2-4970-01,NaN,NaN,3/3,월 34교시 (비409) 화 34교시 (비409) 수 34교시 (비409) 목 34...,NaN,01024224450,...,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 34교시 화 34교시 수 34교시 목 34교시,비409,"(1:3,4),(2:3,4),(3:3,4),(4:3,4)",3
9,공학수학2-Engineering Mathematics 2,2024/3,NaN,0000-2-4970-02,NaN,NaN,3/3,월 56교시 (비409) 화 56교시 (비409) 수 56교시 (비409) 목 56...,NaN,01038605494,...,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 1...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN,월 56교시 화 56교시 수 56교시 목 56교시,비409,"(1:5,6),(2:5,6),(3:5,6),(4:5,6)",3


1. 강의시간 및 강의실 분리  
**** 강의시간 (요일,시간)
**** 강의실 
위와 같이 칼럼 이름 변경 

()안은 강의실을 의미, ()안의 내용이 중복되면 한번만 가져오기 -> '강의실' 칼럼을 새로 만들어 넣어놓기
이외의 텍스트는 '강의시간' 칼럼을 새로 만들어 넣어놓기

2. 학점/시간 중 시간만 분리
**** 학점은 기존 csv에 존재
**** '시간'칼럼 생성
'-' 이후의 숫자인 시간을 분리

3. 교과목명, 년도학기, 이수구분, 담당교수, 학점/시간, 강의시간 및 강의실을 제외하고 위의 csv와 concat하기 

In [147]:
# 제외할 칼럼 리스트
columns_to_exclude = ['교과목명', '년도학기', '이수구분', '담당교수', '학점/시간', '강의시간 및 강의실']

# 제외한 나머지 칼럼만 선택
df_final = df.drop(columns=columns_to_exclude)

In [148]:
# 변환된 데이터 저장
df_final.to_csv(r'D:/LectureRecommendationService/data_preprocessing/[강의시간표] kwu-lecture-timetable/24-여름학기/libs/csv/kwu-24-summer-crawling.csv', index=False, encoding='euc-kr')

* 폐강강의 그대로두기
* 이러닝 -> 학정번호/(강의명도 다른거 있음) 수동작성
* 칼럼 삭제 후 concat

1. 이러닝 강의명 수정

기업경영과비즈니스전략 0000-1-6997-01
상상과창조  0000-1-5706-01
채움과비움:나이듦의인문학적이해 0000-2-9266-01
문화행정론:문화예술로행복한나라만들기  0000-1-4747-01
빅데이터시대의ESG트렌드  0000-2-4655-01
언어의품격:언어바르고품위있게사용하기  0000-2-4656-01
문화콘텐츠스토리텔링전략  0000-1-6061-01
문화기술과사회변동  0000-1-6581-01
저작권과스마트폰의이해  0000-1-8129-01
미래소비사회와상품  0000-1-3535-01
21세기기업의인재상  0000-1-7000-01
색채심리와현대생활  0000-2-6999-01
글로벌시대의예술과가치  0000-2-6580-01
생활속의계약과협상  0000-1-7475-01
전략적의사결정과문제해결  0000-2-7476-01
행복한금융생활  0000-1-9469-01
화폐와금융의과거,현재그리고미래  0000-1-9930-01
음식과세계문화  0000-2-6237-01
그린IT의이해  0000-2-6238-01
